In [2]:
import numpy as np
import sys

import torch
from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
from torch.utils import data, model_zoo
# from model.deeplab_multi import Res_Deeplab
from model.deeplab_single_add_edge import Res_Deeplab

# from dataset.cityscapes_dataset import cityscapesDataSet
from dataset.my_test_dataloader import MyDataSet
from collections import OrderedDict
import os
from PIL import Image

import matplotlib.pyplot as plt
import torch.nn as nn

# config

In [4]:
# DATA_DIRECTORY = './data/Cityscapes/data'
# DATA_LIST_PATH = './dataset/cityscapes_list/val.txt'
# SAVE_PATH = './result/cityscapes'
DATA_DIRECTORY = 'demo_image'
DATA_LIST_PATH = './demo_image/list.txt'
SAVE_PATH = './demo_image_result'

IGNORE_LABEL = 255
NUM_CLASSES = 19
# RESTORE_FROM = './snapshots/GTA2Cityscapes_multi/GTA5_30000.pth'
RESTORE_FROM = '/home/xiaosean/data/20181022-AdaptSeg-2DA-Gated-SA-250k-acc4643/snapshots/GTA2Cityscapes_multi/GTA5_250000.pth'
SET='val'

In [5]:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)
palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)


def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask

In [6]:
testloader = data.DataLoader(MyDataSet(DATA_DIRECTORY, DATA_LIST_PATH, crop_size=(1024, 512), mean=IMG_MEAN, scale=False, mirror=False, set=SET),
                                    batch_size=1, shuffle=False, pin_memory=True)

In [7]:
model = Res_Deeplab(num_classes=19)

In [8]:
saved_state_dict = torch.load(RESTORE_FROM)
model.load_state_dict(saved_state_dict)

In [9]:
model.eval()
model.cuda(0)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [10]:
interp = nn.Upsample(size=(1024, 2048), mode='bilinear')

In [11]:
testloader_iter = iter(testloader)

In [18]:
import torchvision.transforms as transforms
def tensor_to_PIL(tensor):
    pilTrans = transforms.ToPILImage()
    image = pilTrans(tensor.cpu().squeeze(0))
    return image

In [ ]:
with torch.no_grad():
    for index, batch in enumerate(testloader):
        if index % 100 == 0:
            print('%d processd' % index)
        image, _, _, name = batch
        # output1, output2 = model(Variable(image, volatile=True).cuda(gpu0))
        output1, edge = model(Variable(image).cuda(0))

        output = interp(output1).cpu().data[0].numpy()
        edge = interp(edge)
        edge = nn.Sigmoid()(edge).detach()

        output = output.transpose(1,2,0)
        output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

        output_col = colorize_mask(output)
        output = Image.fromarray(output)

        name = name[0].split('/')[-1]
        output.save('%s/%s' % (SAVE_PATH, name))
        output_col.save('%s/%s_color.png' % (SAVE_PATH, name.split('.')[0]))
        tensor_to_PIL(edge).save('%s/%s_edge.png' % (SAVE_PATH, name.split('.')[0]))

print("finish!")

0 processd


/home/xiaosean/anaconda3/envs/xiao/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/xiaosean/anaconda3/envs/xiao/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
